<a href="https://colab.research.google.com/github/jeanlucla/collab/blob/master/Test_JLL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install GPUtil

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil


In [0]:
# Import os to set the environment variable CUDA_VISIBLE_DEVICES
import os
import tensorflow as tf
import GPUtil

In [4]:
# Set CUDA_DEVICE_ORDER so the IDs assigned by CUDA match those from nvidia-smi
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

# Get the first available GPU
DEVICE_ID_LIST = GPUtil.getFirstAvailable()
DEVICE_ID = DEVICE_ID_LIST[0] # grab first element from list
print ('Device List :' ,DEVICE_ID_LIST)


# Set CUDA_VISIBLE_DEVICES to mask out all other GPUs than the first available device id
os.environ["CUDA_VISIBLE_DEVICES"] = str(DEVICE_ID)

# Since all other GPUs are masked out, the first available GPU will now be identified as GPU:0
device = '/gpu:0'
print('Device ID (unmasked): ' + str(DEVICE_ID))
print('Device ID (masked): ' + str(0))



Device List : [0]
Device ID (unmasked): 0
Device ID (masked): 0


# Nouvelle section

In [0]:
GPUs = GPUtil.getGPUs()
GPUavailability = GPUtil.getAvailability(GPUs, maxLoad = 
                                         0.5, maxMemory = 0.5, includeNan=False, excludeID=[], excludeUUID=[])
GPUtil.showUtilization(all=False, attrList=None, useOldCode=False)


| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [6]:
# Run a minimum working example on the selected GPU
# Start a session
with tf.Session() as sess:
    # Select the device
    with tf.device(device):
        # Declare two numbers and add them together in TensorFlow
        a = tf.constant(12)
        b = tf.constant(30)
        result = sess.run(a+b)
        print('a+b=' + str(result))

a+b=42


In [7]:
import sys
import ctypes


# Some constants taken from cuda.h
CUDA_SUCCESS = 0
CU_DEVICE_ATTRIBUTE_MULTIPROCESSOR_COUNT = 16
CU_DEVICE_ATTRIBUTE_MAX_THREADS_PER_MULTIPROCESSOR = 39
CU_DEVICE_ATTRIBUTE_CLOCK_RATE = 13
CU_DEVICE_ATTRIBUTE_MEMORY_CLOCK_RATE = 36


def ConvertSMVer2Cores(major, minor):
    # Returns the number of CUDA cores per multiprocessor for a given
    # Compute Capability version. There is no way to retrieve that via
    # the API, so it needs to be hard-coded.
    return {(1, 0): 8,
            (1, 1): 8,
            (1, 2): 8,
            (1, 3): 8,
            (2, 0): 32,
            (2, 1): 48,
            }.get((major, minor), 192)  # 3.0 and above


libnames = ('libcuda.so', 'libcuda.dylib', 'cuda.dll')

for libname in libnames:
    try:
      cuda = ctypes.CDLL(libname)
    except OSError:
      continue
    else:
       break
else:
    raise OSError("could not load any of: " + ' '.join(libnames))

nGpus = ctypes.c_int()
name = b' ' * 100
cc_major = ctypes.c_int()
cc_minor = ctypes.c_int()
cores = ctypes.c_int()
threads_per_core = ctypes.c_int()
clockrate = ctypes.c_int()
freeMem = ctypes.c_size_t()
totalMem = ctypes.c_size_t()

result = ctypes.c_int()
device = ctypes.c_int()
context = ctypes.c_void_p()
error_str = ctypes.c_char_p()


result = cuda.cuInit(0)
if result != CUDA_SUCCESS:
    cuda.cuGetErrorString(result, ctypes.byref(error_str))
    print("cuInit failed with error code %d: %s" % (result, error_str.value.decode()))
    
result = cuda.cuDeviceGetCount(ctypes.byref(nGpus))
if result != CUDA_SUCCESS:
    cuda.cuGetErrorString(result, ctypes.byref(error_str))
    print("cuDeviceGetCount failed with error code %d: %s" % (result, error_str.value.decode()))

print("Found %d device(s)." % nGpus.value)


for i in range(nGpus.value):
    result = cuda.cuDeviceGet(ctypes.byref(device), i)
    if result != CUDA_SUCCESS:
       cuda.cuGetErrorString(result, ctypes.byref(error_str))
       print("cuDeviceGet failed with error code %d: %s" % (result, error_str.value.decode()))

if cuda.cuDeviceGetName(ctypes.c_char_p(name), len(name), device) == CUDA_SUCCESS:
       print("  Name: %s" % (name.split(b'\0', 1)[0].decode()))
if cuda.cuDeviceComputeCapability(ctypes.byref(cc_major), ctypes.byref(cc_minor), device) == CUDA_SUCCESS:
       print("  Compute Capability: %d.%d" % (cc_major.value, cc_minor.value))
if cuda.cuDeviceGetAttribute(ctypes.byref(cores), CU_DEVICE_ATTRIBUTE_MULTIPROCESSOR_COUNT, device) == CUDA_SUCCESS:
       print("  Multiprocessors: %d" % cores.value)
       print("  CUDA Cores: %d" % (cores.value * ConvertSMVer2Cores(cc_major.value, cc_minor.value)))  
if cuda.cuDeviceGetAttribute(ctypes.byref(threads_per_core), CU_DEVICE_ATTRIBUTE_MAX_THREADS_PER_MULTIPROCESSOR, device) == CUDA_SUCCESS:
       print("  Concurrent threads: %d" % (cores.value * threads_per_core.value))
if cuda.cuDeviceGetAttribute(ctypes.byref(clockrate), CU_DEVICE_ATTRIBUTE_CLOCK_RATE, device) == CUDA_SUCCESS:
       print("  GPU clock: %g MHz" % (clockrate.value / 1000.))
if cuda.cuDeviceGetAttribute(ctypes.byref(clockrate), CU_DEVICE_ATTRIBUTE_MEMORY_CLOCK_RATE, device) == CUDA_SUCCESS:
       print("  Memory clock: %g MHz" % (clockrate.value / 1000.))
       

Found 1 device(s).
  Name: Tesla T4
  Compute Capability: 7.5
  Multiprocessors: 40
  CUDA Cores: 7680
  Concurrent threads: 40960
  GPU clock: 1590 MHz
  Memory clock: 5001 MHz
